# System rekomendacji piosenek

## Zacznijmy od importu potrzebnych bibiotek

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import Recommenders as Recommenders
import Evaluation as Evaluation

## Wczytajmy dane na temat utworów

In [ ]:
song_data = pd.read_csv('dane/song_data.csv')

## Przyjrzyjmy się tym danym

In [ ]:
song_data.head(5)

## Wyszukajmy najpopularniejsze piosenki w zbiorze

In [ ]:
song_grouped = song_data.groupby(['song']).agg({'listen_count': 'count'}).reset_index()
grouped_sum = song_grouped.listen_count.sum() # odtworzenia w sumie
song_grouped.listen_count.div(grouped_sum)*100 # procent odtworzeń dla każdego utworu
song_grouped.sort_values(by='listen_count', ascending=False)

## Policzmy użytkowników i piosenki

In [ ]:
users = song_data['user_id'].unique()
len(users)

In [ ]:
songs = song_data['song'].unique()
len(songs)

## Stwórzmy prosty system rekomendacji utworów na bazie ich popularności

In [ ]:
train_data, test_data = train_test_split(song_data, test_size = 0.20, random_state=0)
pop_model = Recommenders.popularity_recommender_py()
pop_model.create(train_data, 'user_id', 'song')

### Potestuj z różnymi numerami użytkowników

In [ ]:
user_id = users[0] # wpisz w nawias [] liczbę od 0 do 66345
pop_model.recommend(user_id)

## Zbudujmy spersonalizowany system rekomendacji

In [ ]:
is_model = Recommenders.item_similarity_recommender_py()
is_model.create(train_data, 'user_id', 'song')

### Potestuj z różnymi numerami użytkowników

In [ ]:
user_id = users[0] # wpisz w nawias [] liczbę od 0 do 66345
user_items = is_model.get_user_items(user_id)

for user_item in user_items:
    print(user_item)
    
is_model.recommend(user_id)

### Wykorzystajmy model do znalezienia podobnych piosenek

In [ ]:
#song = 'Yellow - Coldplay'
song = 'With Or Without You - U2'
#song = 'Chan Chan (Live) - Buena Vista Social Club'

is_model.get_similar_items([song])

## Porównajmy oba modele

### Procent użytkowników, dla których wyznaczone zostaną metryki modeli

In [ ]:
user_sample = 0.05

### Inicjujemy kalkulator precyzji i czułości

In [ ]:
pr = Evaluation.precision_recall_calculator(test_data, train_data, pop_model, is_model)
(pm_avg_precision_list, pm_avg_recall_list, ism_avg_precision_list, ism_avg_recall_list) = pr.calculate_measures(user_sample)

### Narysujmy krzywe precyzji i czułości

In [ ]:
import pylab as pl

#Method to generate precision and recall curve
def plot_precision_recall(m1_precision_list, m1_recall_list, m1_label, m2_precision_list, m2_recall_list, m2_label):
    pl.clf()    
    pl.plot(m1_recall_list, m1_precision_list, label=m1_label)
    pl.plot(m2_recall_list, m2_precision_list, label=m2_label)
    pl.xlabel('Recall')
    pl.ylabel('Precision')
    pl.ylim([0.0, 0.20])
    pl.xlim([0.0, 0.20])
    pl.title('Precision-Recall curve')
    #pl.legend(loc="upper right")
    pl.legend(loc=9, bbox_to_anchor=(0.5, -0.2))
    pl.show()

In [ ]:
print("Plotting precision recall curves.")

plot_precision_recall(pm_avg_precision_list, pm_avg_recall_list, "popularity_model",
                      ism_avg_precision_list, ism_avg_recall_list, "item_similarity_model")